In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import glob

s = requests.Session()
CATEGORIES_URL = 'https://www.udemy.com/'
COURSES_URL = 'https://www.udemy.com/api-2.0/discovery-units/all_courses'
page = s.get(CATEGORIES_URL)
soup = BeautifulSoup(page.content, 'html.parser')

categories = soup.findAll('a', attrs={'class': 'js-side-nav-cat js-subcat'})

for category in tqdm(categories):
    subcategory_id = category.get('data-id')
    subcategory_name = category.get('href').split('/')[3]
    category_name = category.get('href').split('/')[2]
    file_path = f"data/{category_name}-{subcategory_name}.csv"
    is_exist = os.path.exists(file_path)
    if is_exist:
        continue
    result = pd.DataFrame()
    count = 1
    while True:
        print(count)
        response = s.get(COURSES_URL, params={'p': count,
                                              'page_size': 60,
                                              'subcategory_id': subcategory_id,
                                              'source_page': 'subcategory_page',
                                              'locale': 'en_US',
                                              'currency': 'en_US',
                                              'navigation_locale': 'en_US',
                                              'skip_price': 'true',
                                              'sos': 'ps',
                                              'fl': 'scat'})
        
        if response.status_code == 200:
            if len(response.json()['unit']['items']) == 0 :
                break
        elif response.status_code == 400:
            if response.json() == {'detail': 'Invalid page size'}:
                break

        data = pd.DataFrame.from_dict(data = response.json()['unit']['items'])
        data['category_name'] = category_name
        data['subcategory_id'] = subcategory_id
        data['subcategory_name'] = subcategory_name

        result = pd.concat([result, data])
        
        count += 1
    
    result.to_csv(file_path, index=False)

path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "data/*.csv"))
li = []
# loop over the list of csv files
for file in csv_files:
    df = pd.read_csv(file, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)
data.to_csv('course.csv', index=False)

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import glob
import traceback

s = requests.Session()
PRICE_URL = 'https://www.udemy.com/api-2.0/pricing/'
data = pd.read_csv("/Users/bossthanison/Documents/Portfoilio/project/udemy-analytics/dbt_udemy/seeds/raw_course.csv")

course_ids = list(set(data['id']))
course_ids = [course_ids[i:i+50] for i in range(0,len(course_ids),50)]
result = pd.DataFrame()
fail_list = []
for course_id in tqdm(course_ids):
    try:
        course_id_batch = ",".join(map(str,course_id))
        response = s.get(PRICE_URL, 
                        params={'course_ids': course_id_batch,
                                'fields[pricing_result]': 'price,discount_price,list_price,price_detail,price_serve_tracking_id'})
        df = pd.DataFrame.from_dict(response.json()['courses'])
        df = df.swapaxes("index", "columns")
        result = pd.concat([result, df])
    except:
        fail_list.append(course_id)
        print(f"faile")
  
result.to_csv("price.csv")

 78%|███████▊  | 3337/4287 [2:47:39<5:17:29, 20.05s/it]   

faile


 93%|█████████▎| 4000/4287 [3:23:54<1:33:15, 19.50s/it]

faile


100%|██████████| 4287/4287 [3:40:07<00:00,  3.08s/it]  
